In [ ]:
import keras


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
import string
import re
import joblib
import json
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
from tqdm import tqdm
import os
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [2]:
main_path="C:/Users/akash/Desktop/UI/Chatbot/"
from tensorflow.keras.models import load_model
model = load_model('C:/Users/akash/Desktop/UI/saved_models/rmsmodel3.h5', custom_objects={"BertModelLayer": bert.BertModelLayer})

In [3]:
bert_model_name="uncased_L-12_H-768_A-12"
 
#bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(main_path,"bert_model.ckpt.index")
bert_config_file = os.path.join(main_path,"bert_config.json")

In [4]:
train = pd.read_csv(main_path+"newtrain.csv")
#valid = pd.read_csv("valid.csv")
test = pd.read_csv(main_path+"newtrain.csv")

In [5]:
class IntentDetectionData:
  DATA_COLUMN = "questions"
  LABEL_COLUMN = "labels"
 
  def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])
 
    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])
 
  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))
 
    return np.array(x), np.array(y)
 
  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [6]:
tokenizer = FullTokenizer(vocab_file=os.path.join(main_path+"uncased_L-12_H-768_A-12/vocab.txt"))

In [7]:
classes = train.labels.unique().tolist()
 
data = IntentDetectionData(train, test, tokenizer, classes, max_seq_len=128)

198it [00:00, 1147.94it/s]
C:\Users\akash\anaconda3\lib\site-packages\ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
198it [00:00, 1711.44it/s]

max seq_len 17


In [8]:
# import our chat-bot intents file
import json
with open(main_path+'newintents2.json') as json_data:
    intents = json.load(json_data)

In [9]:
import random
# create a data structure to hold user context
context = {}
 
 
ERROR_THRESHOLD = 0.25
def classify(sentence):
    sentence=[sentence]
    pred_tokens = map(tokenizer.tokenize, sentence)
    pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
    pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
 
    pred_token_ids = map(lambda tids: tids +[0]*(data.max_seq_len-len(tids)),pred_token_ids)
    pred_token_ids = np.array(list(pred_token_ids))
 
    predictions = model.predict(pred_token_ids)
    predictions1=np.argmax(model.predict(pred_token_ids))
    
    return_list =[]
    return_list.append((classes[predictions1],np.amax(predictions[0])))
    
    return return_list
 
 
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    but=[]
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    x=i['id']
                    but=i['buttons']
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: 
                            return (('context:', i['context_set']),x,but)
                        context[userID] = i['context_set']
 
                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']),x,but)
    
            results.pop(0)

In [10]:
sent="centers"
print(classify(sent),response(sent))

[('bvoc_dep_sen', 0.99999976)] ('We offer the following departments to our senior students. Please choose by clicking on one of the buttons', 27, ['Green House Management', 'Pharma Analytical Sciences', 'Tourism Travel Management'])


In [ ]:
from flask import Flask, render_template, request, jsonify
#from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder=main_path+'templates/UI')
#run_with_ngrok(app)

@app.route('/')
def index():
    return render_template('index3.html')

@app.route('/get')
def get_bot_response():
    message = request.args.get('msg')
    print(message)
    if message:
        message = message.lower()
        res,x,buttons=response(message)
        mess = {'response':res,'id':x,'buttons':buttons}
        return jsonify(mess)
    return "Missing Data!"



if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2021 18:48:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2021 18:48:16] "GET /favicon.ico HTTP/1.1" 404 -


what can you do?


127.0.0.1 - - [23/Feb/2021 18:48:24] "GET /get?msg=what%20can%20you%20do%3F HTTP/1.1" 200 -


About Ruhia


...

127.0.0.1 - - [23/Feb/2021 18:56:53] "GET /get?msg=Whatsapp HTTP/1.1" 200 -


Phone


127.0.0.1 - - [23/Feb/2021 18:56:55] "GET /get?msg=Phone HTTP/1.1" 200 -


Email


127.0.0.1 - - [23/Feb/2021 18:56:57] "GET /get?msg=Email HTTP/1.1" 200 -
